In [36]:
from scipy.optimize import minimize
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import yfinance as yf
from backtest import *

In [108]:
# Set the start and end date
start_date = '2000-01-01'
end_date = '2022-12-31'

# Set the tickers
tickers = ["SPY5.L", "SGLN.L", "XDWT.DE"]
risk_free = yf.download('^IRX', start = start, end = end)['Adj Close']
risk_free = float(risk_free.tail(1))

# Get the data
data = yf.download(tickers, start_date, end_date)
data = data["Adj Close"]

# Drop missing values
data = data.dropna().pct_change(1).dropna()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  3 of 3 completed


In [90]:
def MV_criterion(weights, data):
    """
    -----------------------------------------------------------------------------
    | Output: optimization porfolio criterion                                   |
    -----------------------------------------------------------------------------
    | Inputs: -weight (type ndarray numpy): Weight for portfolio               |
    |         -data (type ndarray numpy): Returns of stocks                     |
    -----------------------------------------------------------------------------
    """

    # Parameters
    Lambda = 3
    W = 1
    Wbar = 1 + risk_free / 100

    # Compute portfolio returns
    portfolio_return = np.multiply(data, np.transpose(weights))
    portfolio_return = portfolio_return.sum(axis=1)

    # Compute mean and volatility of the portfolio
    mean = np.mean(portfolio_return, axis=0)
    std = np.std(portfolio_return, axis=0)

    # Compute the criterion
    criterion = Wbar ** (1 - Lambda) / (1 + Lambda) + Wbar ** (-Lambda) \
                * W * mean - Lambda / 2 * Wbar ** (-1 - Lambda) * W ** 2 * std ** 2
    criterion = -criterion
    
    return criterion

In [107]:
# Define train and test sets
split = int(0.8 * len(data))
train_set = data.iloc[:split, :]
test_set = data.iloc[split:, :]

# Find the number of asset
n = data.shape[1]

# Initialisation weight value
x0 = np.ones(n)

# Optimization constraints problem
cons = ({'type': 'eq', 'fun': lambda x: sum(abs(x)) - 1})

# Set the bounds
Bounds = [(0, 1) for i in range(0, n)]

# Optimization problem solving
res_MV = minimize(MV_criterion, x0, method="SLSQP",
                  args=(train_set), bounds=Bounds,
                  constraints=cons, options={'disp': False})

# Result for computations
X_MV = res_MV.x

print(X_MV)

[0.33331761 0.33325123 0.33343117]
